# Project Summary
## Problem/goals
The goal of this kaggle project is to detect external contact experienced by players during an NFL football game. You will use video and player tracking data to identify moments with contact to help improve player safety. 
The NFL aspire to have best injury surveillance and mitigation program. The Machine learning and computer visons enabled method can help  to accurately identify when the player may experience contact throughout football play.

## Dataset
Video clips of 29 plays have been provided. In addition, metadata to identify helmets and tracking player's position during the play. We need to provide prediciton for 61 plays hiddnen in the test data. We need to predict whether player experience contact (1) or no contact (1) with ground or another player.

## Methods
Following strategies used.
* frames from each plays are extracted. The metadata was implemented. 
* images were augmented
* model trained
* images data are synced with metadata
* created a pipeline to preprocess and input test data (hidden) into the model for evaluation.
* CNN Model architecture:

## Challanges
* creating box to detect helmets was challanging.

## Next steps
*

## Outcomes
* LB score:0.67.
* successfully predicted the contact for player.

In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import random
import math
import gc
import cv2
from tqdm import tqdm
import time
from functools import lru_cache
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from timm.scheduler import CosineLRScheduler
sys.path.append('../input/timm-0-6-9/pytorch-image-models-master')

# Config

In [2]:
CFG = {
    'seed': 42,
    'model': 'resnet50',
    'img_size': 256,
    'epochs': 10,
    'train_bs': 8, 
    'valid_bs': 4,
    'lr': 1e-3, 
    'weight_decay': 1e-6,
    'num_workers': 8,
    'max_grad_norm' : 1000,
    'epochs_warmup' : 1.0
}

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['seed'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# EDA

In [4]:
train_labels= pd.read_csv("../input/nfl-player-contact-detection/train_labels.csv")
train_labels.head()

,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact
0,58168_003392_0_38590_43854,58168_003392,2020-09-11T03:01:48.100Z,0,38590,43854,0
1,58168_003392_0_38590_41257,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41257,0
2,58168_003392_0_38590_41944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41944,0
3,58168_003392_0_38590_42386,58168_003392,2020-09-11T03:01:48.100Z,0,38590,42386,0
4,58168_003392_0_38590_47944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,47944,0


In [17]:
# Investigate steps
(train_labels
 .groupby('game_play')
 .step
 .agg(['count',min,max,'mean'])
 .reset_index()
 .sort_values('max',ascending=False))
# The step ranges from 0 to upto 172. 

,game_play,count,min,max,mean
195,58537_000757,43769,0,172,86.0
181,58524_002909,41492,0,163,81.5
54,58224_003139,34155,0,134,67.0
2,58173_003606,32890,0,129,64.5
1,58172_003247,31878,0,125,62.5
...,...,...,...,...,...
216,58555_002451,11638,0,45,22.5
186,58526_001849,9614,0,37,18.5
220,58558_000573,9614,0,37,18.5
110,58316_004343,9614,0,37,18.5


#### Contact

In [33]:
# No of contacts per game.
(train_labels[train_labels.contact==1]
 .groupby('game_play')
 .contact
 .count()
 .reset_index()
 .sort_values('contact',ascending=False)
 .head(10)
)
# Max no. of contacts/game: 750

,game_play,contact
208,58551_003569,750
102,58308_004092,733
33,58204_002864,709
42,58216_001939,688
104,58311_002159,682
173,58516_003538,663
194,58537_000757,652
161,58507_003903,609
4,58176_002844,596
75,58270_002527,594


In [21]:
# Investigate contact

(train_labels
 .groupby(['game_play','step','contact'])
 .contact
 .agg(['count','mean'])
 .reset_index()
)

,game_play,step,contact,count,mean
0,58168_003392,0,0,253,0.0
1,58168_003392,1,0,253,0.0
2,58168_003392,2,0,253,0.0
3,58168_003392,3,0,252,0.0
4,58168_003392,3,1,1,1.0
...,...,...,...,...,...
32910,58582_003121,89,1,1,1.0
32911,58582_003121,90,0,252,0.0
32912,58582_003121,90,1,1,1.0
32913,58582_003121,91,0,252,0.0


In [10]:
train_labels.step.min(),train_labels.step.max()
# This means game 

(0, 172)

**Insight**:  
* Contact_id is constructed by joining: game_play+step+nfl_player_id_1+nfl_player_id_2
* step: is time in sec after the start. 0 means in 0th sec after game start.



In [5]:
train_tracking = pd.read_csv("../input/nfl-player-contact-detection/train_player_tracking.csv")
train_tracking.head()

,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,y_position,speed,distance,direction,orientation,acceleration,sa
0,58580_001136,58580,1136,44830,2021-10-10T21:08:20.900Z,-108,away,CB,22,61.59,42.60,1.11,0.11,320.33,263.93,0.71,-0.64
1,58580_001136,58580,1136,47800,2021-10-10T21:08:20.900Z,-108,away,DE,97,59.48,26.81,0.23,0.01,346.84,247.16,1.29,0.90
2,58580_001136,58580,1136,52444,2021-10-10T21:08:20.900Z,-108,away,FS,29,72.19,31.46,0.61,0.06,11.77,247.69,0.63,-0.33
3,58580_001136,58580,1136,46206,2021-10-10T21:08:20.900Z,-108,home,TE,86,57.37,22.12,0.37,0.04,127.85,63.63,0.69,0.62
4,58580_001136,58580,1136,52663,2021-10-10T21:08:20.900Z,-108,away,ILB,48,63.25,27.50,0.51,0.05,183.62,253.71,0.31,0.31


In [6]:
train_helmets = pd.read_csv("../input/nfl-player-contact-detection/train_baseline_helmets.csv")
train_helmets.head()

,game_play,game_key,play_id,view,video,frame,nfl_player_id,player_label,left,width,top,height
0,58168_003392,58168,3392,Endzone,58168_003392_Endzone.mp4,290,39947,H72,946,25,293,34
1,58168_003392,58168,3392,Endzone,58168_003392_Endzone.mp4,290,37211,H42,151,25,267,33
2,58168_003392,58168,3392,Endzone,58168_003392_Endzone.mp4,290,38590,H70,810,25,293,35
3,58168_003392,58168,3392,Endzone,58168_003392_Endzone.mp4,290,44822,H15,681,26,254,33
4,58168_003392,58168,3392,Endzone,58168_003392_Endzone.mp4,290,41944,V92,680,23,303,33


In [7]:
train_video_metadata = pd.read_csv("../input/nfl-player-contact-detection/train_video_metadata.csv")
train_video_metadata.head()

,game_play,game_key,play_id,view,start_time,end_time,snap_time
0,58168_003392,58168,3392,Endzone,2020-09-11T03:01:43.134Z,2020-09-11T03:01:54.971Z,2020-09-11T03:01:48.134Z
1,58168_003392,58168,3392,Sideline,2020-09-11T03:01:43.134Z,2020-09-11T03:01:54.971Z,2020-09-11T03:01:48.134Z
2,58172_003247,58172,3247,Endzone,2020-09-13T19:30:42.414Z,2020-09-13T19:31:00.524Z,2020-09-13T19:30:47.414Z
3,58172_003247,58172,3247,Sideline,2020-09-13T19:30:42.414Z,2020-09-13T19:31:00.524Z,2020-09-13T19:30:47.414Z
4,58173_003606,58173,3606,Endzone,2020-09-13T19:45:07.527Z,2020-09-13T19:45:26.438Z,2020-09-13T19:45:12.527Z


In [8]:
sample_submission=pd.read_csv('/kaggle/input/nfl-player-contact-detection/sample_submission.csv')
sample_submission.head()

,contact_id,contact
0,58168_003392_0_38590_43854,0
1,58168_003392_0_38590_41257,0
2,58168_003392_0_38590_41944,0
3,58168_003392_0_38590_42386,0
4,58168_003392_0_38590_47944,0


# Data Preparation

In [6]:
def expand_contact_id(df):
    """
    Splits out contact_id into seperate columns.
    """
    df["game_play"] = df["contact_id"].str[:12]
    df["step"] = df["contact_id"].str.split("_").str[-3].astype("int")
    df["nfl_player_id_1"] = df["contact_id"].str.split("_").str[-2]
    df["nfl_player_id_2"] = df["contact_id"].str.split("_").str[-1]
    return df

In [7]:
labels = expand_contact_id(pd.read_csv("../input/nfl-player-contact-detection/train_labels.csv"))
train_tracking = pd.read_csv("../input/nfl-player-contact-detection/train_player_tracking.csv")
train_helmets = pd.read_csv("../input/nfl-player-contact-detection/train_baseline_helmets.csv")
train_video_metadata = pd.read_csv("../input/nfl-player-contact-detection/train_video_metadata.csv")

# Create frames

In [ ]:
!mkdir -p ../train/frames

for video in tqdm(train_helmets.video.unique()):
    if 'Endzone2' not in video:
        !ffmpeg -i ../input/nfl-player-contact-detection/train/{video} -q:v 2 -f image2 ../train/frames/{video}_%04d.jpg -hide_banner -loglevel error

# Create features

In [ ]:
def create_features(df, tr_tracking, merge_col="step", use_cols=["x_position", "y_position"]):
    output_cols = []
    df_combo = (
        df.astype({"nfl_player_id_1": "str"})
        .merge(
            tr_tracking.astype({"nfl_player_id": "str"})[
                ["game_play", merge_col, "nfl_player_id",] + use_cols
            ],
            left_on=["game_play", merge_col, "nfl_player_id_1"],
            right_on=["game_play", merge_col, "nfl_player_id"],
            how="left",
        )
        .rename(columns={c: c+"_1" for c in use_cols})
        .drop("nfl_player_id", axis=1)
        .merge(
            tr_tracking.astype({"nfl_player_id": "str"})[
                ["game_play", merge_col, "nfl_player_id"] + use_cols
            ],
            left_on=["game_play", merge_col, "nfl_player_id_2"],
            right_on=["game_play", merge_col, "nfl_player_id"],
            how="left",
        )
        .drop("nfl_player_id", axis=1)
        .rename(columns={c: c+"_2" for c in use_cols})
        .sort_values(["game_play", merge_col, "nfl_player_id_1", "nfl_player_id_2"])
        .reset_index(drop=True)
    )
    output_cols += [c+"_1" for c in use_cols]
    output_cols += [c+"_2" for c in use_cols]
    
    if ("x_position" in use_cols) & ("y_position" in use_cols):
        index = df_combo['x_position_2'].notnull()
        
        distance_arr = np.full(len(index), np.nan)
        tmp_distance_arr = np.sqrt(
            np.square(df_combo.loc[index, "x_position_1"] - df_combo.loc[index, "x_position_2"])
            + np.square(df_combo.loc[index, "y_position_1"]- df_combo.loc[index, "y_position_2"])
        )
        
        distance_arr[index] = tmp_distance_arr
        df_combo['distance'] = distance_arr
        output_cols += ["distance"]
        
    df_combo['G_flug'] = (df_combo['nfl_player_id_2']=="G")
    output_cols += ["G_flug"]
    return df_combo, output_cols


use_cols = [
    'x_position', 'y_position', 'speed', 'distance',
    'direction', 'orientation', 'acceleration', 'sa'
]

train, feature_cols = create_features(labels, train_tracking, use_cols=use_cols)

In [ ]:
train_filtered = train.query('not distance>2').reset_index(drop=True)
train_filtered['frame'] = (train_filtered['step']/10*59.94+5*59.94).astype('int')+1
train_filtered.head()

del train, labels, train_tracking
gc.collect()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

train_aug = ImageDataGenerator(
    horizontal_flip=True,
    brightness_range=(-0.1, 0.1),
    zoom_range=(0.9, 1.1),
    rotation_range=20,
    preprocessing_function=preprocess_input
)

valid_aug = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


### Albumentations 
is a Python library for image augmentation. It provides a wide range of image transformations, such as geometric transformations (e.g., flips, rotations, translations), color manipulations (e.g., brightness, contrast, saturation), and more advanced techniques such as cutout and grid distortion.

In [ ]:

train_aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
    A.Normalize(mean=[0.], std=[1.]),
    ToTensorV2()
])

valid_aug = A.Compose([
    A.Normalize(mean=[0.], std=[1.]),
    ToTensorV2()
])

In [ ]:
video2helmets = {}
train_helmets_new = train_helmets.set_index('video')
for video in tqdm(train_helmets.video.unique()):
    video2helmets[video] = train_helmets_new.loc[video].reset_index(drop=True)

del train_helmets, train_helmets_new
gc.collect()

In [ ]:
video2frames = {}

for game_play in tqdm(train_video_metadata.game_play.unique()):
    for view in ['Endzone', 'Sideline']:
        video = game_play + f'_{view}.mp4'
        video2frames[video] = max(list(map(lambda x:int(x.split('_')[-1].split('.')[0]), \
                                           glob.glob(f'../train/frames/{video}*'))))

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, aug=train_aug, mode='train'):
        self.df = df
        self.frame = df.frame.values
        self.feature = df[feature_cols].fillna(-1).values
        self.players = df[['nfl_player_id_1','nfl_player_id_2']].values
        self.game_play = df.game_play.values
        self.aug = aug
        self.mode = mode
        
    def __len__(self):
        return len(self.df)
    
    # @lru_cache(1024)
    # def read_img(self, path):
    #     return cv2.imread(path, 0)
   
    def __getitem__(self, idx):   
        window = 24
        frame = self.frame[idx]
        
        if self.mode == 'train':
            frame = frame + random.randint(-6, 6)

        players = []
        for p in self.players[idx]:
            if p == 'G':
                players.append(p)
            else:
                players.append(int(p))
        
        imgs = []
        for view in ['Endzone', 'Sideline']:
            video = self.game_play[idx] + f'_{view}.mp4'

            tmp = video2helmets[video]
#             tmp = tmp.query('@frame-@window<=frame<=@frame+@window')
            tmp[tmp['frame'].between(frame-window, frame+window)]
            tmp = tmp[tmp.nfl_player_id.isin(players)]#.sort_values(['nfl_player_id', 'frame'])
            tmp_frames = tmp.frame.values
            tmp = tmp.groupby('frame')[['left','width','top','height']].mean()
#0.002s

            bboxes = []
            for f in range(frame-window, frame+window+1, 1):
                if f in tmp_frames:
                    x, w, y, h = tmp.loc[f][['left','width','top','height']]
                    bboxes.append([x, w, y, h])
                else:
                    bboxes.append([np.nan, np.nan, np.nan, np.nan])
            bboxes = pd.DataFrame(bboxes).interpolate(limit_direction='both').values
            bboxes = bboxes[::4]

            if bboxes.sum() > 0:
                flag = 1
            else:
                flag = 0
#0.03s
                    
            for i, f in enumerate(range(frame-window, frame+window+1, 4)):
                img_new = np.zeros((256, 256), dtype=np.float32)

                if flag == 1 and f <= video2frames[video]:
                    img = cv2.imread(f'../train/frames/{video}_{f:04d}.jpg', 0)

                    x, w, y, h = bboxes[i]

                    img = img[int(y+h/2)-128:int(y+h/2)+128,int(x+w/2)-128:int(x+w/2)+128].copy()
                    img_new[:img.shape[0], :img.shape[1]] = img
                    
                imgs.append(img_new)
#0.06s
                
        feature = np.float32(self.feature[idx])

        img = np.array(imgs).transpose(1, 2, 0)    
        img = self.aug(image=img)["image"]
        label = np.float32(self.df.contact.values[idx])

        return img, feature, label

In [ ]:
# class Model(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.backbone = timm.create_model(CFG['model'], pretrained=True, num_classes=500, in_chans=13)
#         self.mlp = nn.Sequential(
#             nn.Linear(18, 64),
#             nn.LayerNorm(64),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#         )
#         self.fc = nn.Linear(64+500*2, 1)

#     def forward(self, img, feature):
#         b, c, h, w = img.shape
#         img = img.reshape(b*2, c//2, h, w)
#         img = self.backbone(img).reshape(b, -1)
#         feature = self.mlp(feature)
#         y = self.fc(torch.cat([img, feature], dim=1))
#         return y

In [1]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

class MyDataset(tf.keras.utils.Sequence):
    def __init__(self, df, aug=None, mode='train', batch_size=32):
        self.df = df
        self.frame = df.frame.values
        self.feature = df[feature_cols].fillna(-1).values
        self.players = df[['nfl_player_id_1', 'nfl_player_id_2']].values
        self.game_play = df.game_play.values
        self.aug = aug
        self.mode = mode
        self.batch_size = batch_size

    def __len__(self):
        return (len(self.df) + self.batch_size - 1) // self.batch_size

    def __getitem__(self, idx):
        batch = slice(idx * self.batch_size, (idx + 1) * self.batch_size)
        batch_df = self.df.iloc[batch]

        window = 24
        frame = batch_df.frame.values

        if self.mode == 'train':
            frame = frame + tf.random.uniform(batch_df.shape, -6, 6, dtype=tf.int64)

        players = tf.reshape(batch_df[['nfl_player_id_1', 'nfl_player_id_2']].values, (-1, 2))
        players = tf.where(players == 'G', players, tf.strings.to_number(players, tf.int64))

        imgs = []
        for view in ['Endzone', 'Sideline']:
            video = batch_df.game_play.values + f'_{view}.mp4'

            tmp = video2helmets[video]
            tmp = tmp[tmp['frame'].between(frame-window, frame+window)]
            tmp = tmp[tmp.nfl_player_id.isin(players.numpy().ravel())]
            tmp_frames = tmp.frame.values
            tmp = tmp.groupby('frame')[['left', 'width', 'top', 'height']].mean()

            bboxes = []
            for f in range(frame-window, frame+window+1, 1):
                if f in tmp_frames:
                    x, w, y, h = tmp.loc[f][['left', 'width', 'top', 'height']]
                    bboxes.append([x, w, y, h])
                else:
                    bboxes.append([float('nan'), float('nan'), float('nan'), float('nan')])
            bboxes = tf.convert_to_tensor(bboxes, dtype=tf.float32)
            bboxes = tfp.stats.interpolate_nan(bboxes, axis=0, limit_direction='both').numpy()
            bboxes = bboxes[::4]

            flag = tf.reduce_sum(tf.cast(tf.math.is_finite(bboxes), tf.int32))
            flag = tf.where(flag > 0, 1, 0)

            for i, f in enumerate(range(frame-window, frame+window+1, 4)):
                img_new = tf.zeros((256, 256), dtype=tf.float32)

                if flag == 1 and f <= video2frames[video]:
                    img = tf.io.read_file(f'../train/frames/{video}_{f:04d}.jpg')
                    img = tf.image.decode_jpeg(img, channels=1)
                    img = tf.image.crop_to_bounding_box(img, int(bboxes[i][2]+bboxes[i][3]/2)-128, int(bboxes[i][0]+bboxes[i][1]/2)-128, 256, 256)
                    img_new = tf.image.convert_image_dtype(img, dtype=tf.float32)

                imgs.append(img_new)

        feature = tf.convert_to_tensor(batch_df[feature_cols])


SyntaxError: unexpected EOF while parsing (3025817439.py, line 66)

In [ ]:
model = Model()
model.to(device)
model.train()

In [ ]:
import torch.nn as nn
criterion = nn.BCEWithLogitsLoss()

In [ ]:
def evaluate(model, loader_val, *, compute_score=True, pbar=None):
    """
    Predict and compute loss and score
    """
    tb = time.time()
    in_training = model.training
    model.eval()

    loss_sum = 0.0
    n_sum = 0
    y_all = []
    y_pred_all = []

    if pbar is not None:
        pbar = tqdm(desc='Predict', nrows=78, total=pbar)
        
    total= len(loader_val)

    for ibatch,(img, feature, label) in tqdm(enumerate(loader_val),total = total):
        # img, feature, label = [x.to(device) for x in batch]
        img = img.to(device)
        feature = feature.to(device)
        n = label.size(0)
        label = label.to(device)

        with torch.no_grad():
            y_pred = model(img, feature)
        loss = criterion(y_pred.view(-1), label)

        n_sum += n
        loss_sum += n * loss.item()
        
        if pbar is not None:
            pbar.update(len(img))
        
        del loss, img, label
        gc.collect()

    loss_val = loss_sum / n_sum


    ret = {'loss': loss_val,
           'time': time.time() - tb}
    
    model.train(in_training) 
    gc.collect()
    return ret

In [ ]:
train_set,valid_set = train_test_split(train_filtered,test_size=0.05, random_state=42,stratify = train_filtered['contact'])
train_set = MyDataset(train_set, train_aug, 'train')
train_loader = DataLoader(train_set, batch_size=CFG['train_bs'], shuffle=True, num_workers=12, pin_memory=True,drop_last=True)
valid_set = MyDataset(valid_set, valid_aug, 'test')
valid_loader = DataLoader(valid_set, batch_size=CFG['valid_bs'], shuffle=False, num_workers=12, pin_memory=True)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG['lr'], weight_decay=CFG['weight_decay'])
nbatch = len(train_loader)
warmup = CFG['epochs_warmup'] * nbatch
nsteps = CFG['epochs'] * nbatch 

In [ ]:
scheduler = CosineLRScheduler(optimizer,warmup_t=warmup, warmup_lr_init=0.0, warmup_prefix=True,t_initial=(nsteps - warmup), lr_min=1e-6)                

In [ ]:
time_val = 0.0
tb = time.time()
best_cv = 0
best_loss = 1e10
for iepoch in range(CFG['epochs']):
    print('Epoch:', iepoch+1)
    loss_sum = 0.0
    n_sum = 0
    total = len(train_loader)

    # Train
    for ibatch,(img, feature, label) in tqdm(enumerate(train_loader),total = total):
        img = img.to(device)
        feature = feature.to(device)
        n = label.size(0)
        label = label.to(device)
        

        optimizer.zero_grad()
        y_pred = model(img, feature).squeeze(-1)
        loss = criterion(y_pred, label)
        loss_train = loss.item()
        loss_sum += n * loss_train
        n_sum += n

        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(),CFG['max_grad_norm'])

        optimizer.step()
        scheduler.step(iepoch * nbatch + ibatch + 1)
        
    val = evaluate(model, valid_loader)
    time_val += val['time']
    loss_train = loss_sum / n_sum
    dt = (time.time() - tb) / 60
    print('Epoch: %d Train Loss: %.4f Test Loss: %.4f Time: %.2f min' %
          (iepoch + 1, loss_train, val['loss'],dt))
    if val['loss'] < best_loss:
        best_loss = val['loss']
        # Save model
        ofilename = 'best_model.pytorch'
        torch.save(model.state_dict(), ofilename)
        print(ofilename, 'written')
    del val
    gc.collect()

dt = time.time() - tb
print(' %.2f min total, %.2f min val' % (dt / 60, time_val / 60))
gc.collect()